In [1]:
%load_ext autotime
from arc.species.species import ARCSpecies
from arc.species.zmat import zmat_to_coords
from arc.species.converter import xyz_to_xyz_file_format, str_to_zmat, zmat_to_xyz, modify_coords
from arc.plotter import show_sticks
import yaml
import os
import psi4
import shutil
from itertools import combinations 
from joblib import Parallel, delayed
from typing import Dict, Iterable, List, Optional, Tuple, Union
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def convert_gaussian_zmat_to_arc_zmat(zmat_file_path):
# Read Gaussian input file with zmat info, output arc zmat

    # read gaussian input
    atom_dict = dict()
    hydrogen_list = list()
    connectivity_dict = dict()

    with open(zmat_file_path, 'r') as f:
        index = 1
        line = f.readline()
        flag0 = True
        flag1 = False
        flag2 = False
        while line:
            if flag0:
                try:
                    if not line.split()[0] == "0":
                        line = f.readline()
                    else:
                        flag0 = False
                        flag1 = True
                        line = f.readline()
                except IndexError:
                    line = f.readline()

            if flag1:
                line_content = line.split()
                atom_dict[index] = dict()
                atom_dict[index].update(atom = line_content[0])
                if line_content[0] == 'H':
                    hydrogen_list.append(index)
                if index > 1:
                    atom_dict[index].update(bond = (line_content[1], line_content[2]))
                if index > 2:
                    atom_dict[index].update(angle = (line_content[3], line_content[4]))
                if index > 3:
                    atom_dict[index].update(dihedral = (line_content[5], line_content[6]))
                line = f.readline()
                index += 1
                if not line.split():
                    flag1 = False
                    flag2 = True
                    line = f.readline()
            elif flag2:
                line_content = line.split()
                if not line_content:
                    break
                key = line_content[0]
                val = line_content[1]
                connectivity_dict[key] = val
                line = f.readline()
      
    # convert to ARC zmat
    arc_zmat = dict()
    symbols_list = list()
    coords_list = list()
    vars_dict = dict()
    map_dict = dict()
            
    for atom_id in atom_dict.keys():
        atom_num = atom_id - 1
        map_dict[atom_num] = atom_num

        atom_id_dict = atom_dict[atom_id]

        symbols_list.append(atom_id_dict['atom'])

        bond_info_tuple = atom_id_dict.get('bond', None)
        angle_info_tuple = atom_id_dict.get('angle', None)
        dihedral_info_tuple = atom_id_dict.get('dihedral', None)

        R = None
        A = None
        D = None

        if bond_info_tuple is not None:
            r = str(int(bond_info_tuple[0]) - 1)
            R = '_'.join(['R', str(atom_num), r])
            vars_dict[R] = float(connectivity_dict[bond_info_tuple[1]])

        if angle_info_tuple is not None:
            a = str(int(angle_info_tuple[0]) - 1)
            A = '_'.join(['A', str(atom_num), r, a])
            vars_dict[A] = float(connectivity_dict[angle_info_tuple[1]])

        if dihedral_info_tuple is not None:
            d = str(int(dihedral_info_tuple[0]) - 1)
            D = '_'.join(['D', str(atom_num), r, a, d])
            vars_dict[D] = float(connectivity_dict[dihedral_info_tuple[1]])

        coords_list.append((R, A, D))

    arc_zmat['symbols'] = tuple(symbols_list)
    arc_zmat['coords'] = tuple(coords_list)
    arc_zmat['vars'] = vars_dict
    arc_zmat['map'] = map_dict
    
    return arc_zmat

time: 4.63 ms


In [3]:
cwd = os.getcwd()
zmat_file_path = os.path.join(cwd, 'imipramine_4_oo.gjf')
arc_zmat = convert_gaussian_zmat_to_arc_zmat(zmat_file_path)

time: 1.73 ms


In [4]:
spc_name = 'imipramine_4_oo'
spc = ARCSpecies(label=spc_name, xyz=arc_zmat, is_ts=False, multiplicity=2, smiles='NCCC(O[O])N(C)C')
spc.mol_from_xyz(arc_zmat)

spc.determine_rotors()
dihedrals = list()
for i in range(len(spc.rotors_dict.keys())):
    dihedrals.append(spc.rotors_dict[i]['scan'])

comb = combinations(dihedrals, 2)
all_2d_dihedrals = tuple(comb)

cwd = os.getcwd()
save_folder = cwd + '/' + spc_name + '_2d_grid_sp'
if os.path.exists(save_folder):
    shutil.rmtree(save_folder)
os.mkdir(save_folder)

scan_res = 45 # 360 / 45 = 8 deg per scan

time: 517 ms


In [5]:
def wrapper(n):
    xyz_new_dict = dict()
    indices_a = all_2d_dihedrals[n][0]
    indices_b = all_2d_dihedrals[n][1]
    print('-----------------------------')
    print(f'considering combination: {all_2d_dihedrals[n]}')
    for i in range(scan_res):
        zmat_1 = modify_coords(coords=arc_zmat,
                          indices=indices_a,
                          new_value=i*8,
                          modification_type='groups',
                          mol=spc.mol,
                          index=1,
                          output_zmat = True,
                         )
        
        for j in range(scan_res):
            zmat_2 = modify_coords(coords=zmat_1,
                                  indices=indices_b,
                                  new_value=j*8,
                                  modification_type='groups',
                                  mol=spc.mol,
                                  index=1,
                                  output_zmat = True,
                                 )
            
            xyz_new_dict[(i,j)] = xyz_to_xyz_file_format(zmat_to_xyz(zmat_2))
            
    comb_name = '_'.join([str(elem) for elem in indices_a]) + '_n_' + '_'.join([str(elem) for elem in indices_b])
    file_name = spc_name + '_' + comb_name + '_coord.yml'
    file_path = save_folder + '/' + file_name
    with open(file_path, 'w') as outfile:
        yaml.dump(xyz_new_dict, outfile, default_flow_style=False)

time: 1.6 ms


In [6]:
Parallel(n_jobs=-1)(delayed(wrapper)(n) for n in range(len(all_2d_dihedrals)))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

time: 1min 25s
